In [0]:
import numpy as np
import pandas as pd
import time
import warnings
import traceback
import sys
import os
from sklearn.ensemble import RandomForestRegressor
from joblib import Parallel, delayed
from scipy.stats import pearsonr, spearmanr
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor
import matplotlib.pyplot as plt

In [0]:
# --- RUTA DEL ARCHIVO ---
# [TFM NOTE]: Ruta hardcodeada para pruebas. En producción usar args o env vars.
RUTA = "abfss://datos@mastertfm001sta.dfs.core.windows.net/gold/config/GOLD_Acciones_2025.parquet"

estrategias_path = (
    "abfss://datos@mastertfm001sta.dfs.core.windows.net/"
    "gold/activos/config/estrategias_optimas.csv"
)

In [0]:
# =========================================================
# CONFIGURACIÓN DE ACELERACIÓN (JIT - NUMBA)
# [TFM NOTE]: Se utiliza compilación Just-In-Time (JIT) para traducir
# la lógica financiera de Python a Código Máquina optimizado.
# =========================================================
try:
    from numba import jit
    HAS_NUMBA = True
    print("✅ Aceleración Numba activada.")
except ImportError:
    HAS_NUMBA = False
    print("⚠️ Numba no instalado. Usando modo Python estándar (más lento).")
    def jit(*args, **kwargs):
        def decorator(func):
            return func
        return decorator

⚠️ Numba no instalado. Usando modo Python estándar (más lento).


In [0]:
# =========================================================
# 1. NÚCLEO MATEMÁTICO (Compilado para velocidad)
# =========================================================

@jit(nopython=True)
def _signed_moves_with_threshold(precios, j):
    """
    Calcula la señal de movimiento discretizada $S_t$.
    [TFM NOTE]: Discretización para filtrar ruido de microestructura (J).
    """
    n = len(precios)
    s = np.zeros(n - 1, dtype=np.int8)
    for i in range(n - 1):
        p_t = precios[i]
        if p_t <= 1e-9:
            s[i] = 0
            continue
        r = precios[i+1] / p_t - 1.0
        if r >= j: s[i] = 1
        elif r <= -j: s[i] = -1
    return s

@jit(nopython=True)
def _calc_pnl_ops(precios, k, n, j):
    """
    Simula la estrategia BASE (Inercia).
    Retorna (pnl, ops).
    [TFM NOTE]: Motor de Backtesting Vectorizado.
    """
    n_samples = len(precios)
    if n_samples < k + n + 2: return np.nan, 0

    direcciones = _signed_moves_with_threshold(precios, j)
    pnl = 0.0
    ops = 0
    t = k

    while t + n < n_samples:
        # Lógica de Patrón (Unanimidad en K periodos)
        es_compra = True
        es_venta = True
        for i in range(1, k + 1):
            senal = direcciones[t - i]
            if senal != 1: es_compra = False
            if senal != -1: es_venta = False
            if not es_compra and not es_venta: break

        p_entry = precios[t]
        if p_entry <= 1e-9:
            t += 1
            continue

        r = precios[t + n] / p_entry - 1.0

        if es_compra:
            pnl += r
            ops += 1
            t += n
        elif es_venta:
            pnl += -r
            ops += 1
            t += n
        else:
            t += 1

    if ops > 0: return pnl, ops
    else: return np.nan, 0

def evaluate_strategy_wrapper(params_pack, cache, train_ratio, test_ratio, h_asset, lam):
    """
    Worker Inteligente: Evalúa Inercia y Rebote simultáneamente.
    [TFM NOTE]: Función serializable para distribución en núcleos CPU.
    """
    try:
        x, k, n, j = params_pack
        if int(x) not in cache: return None
        tr_x, te_x = cache[int(x)]
        if tr_x.empty or te_x.empty: return None

        p_tr = tr_x["close"].values
        p_te = te_x["close"].values

        # 1. Simulación Train
        tr_ret, tr_ops = _calc_pnl_ops(p_tr, int(k), int(n), float(j))
        if tr_ops == 0 or np.isnan(tr_ret): return None

        # 2. Simulación Test
        te_ret, _ = _calc_pnl_ops(p_te, int(k), int(n), float(j))

        if train_ratio <= 1e-9 or test_ratio <= 1e-9: return None

        g_tr = tr_ret / train_ratio
        g_te = te_ret / test_ratio if not np.isnan(te_ret) else np.nan

        # Validación de Consistencia (Evitar Overfitting por azar)
        if np.isnan(g_te) or np.sign(g_tr) != np.sign(g_te): return None

        # --- LÓGICA DUAL (INERCIA vs REBOTE) ---
        delta = abs(g_tr - g_te)
        cost = (tr_ops / train_ratio) * h_asset

        score = -1.0

        if g_tr > 0:
            # CASO 1: INERCIA (Trend Following)
            base = g_tr - lam * delta
            score = base - cost
        else:
            # CASO 2: REBOTE (Mean Reversion)
            # Si pierde dinero consistentemente, operar a la contra es rentable.
            g_tr_inv = -g_tr
            base = g_tr_inv - lam * delta
            score = base - cost

        if score > 0:
            return [x, k, n, j, score]

        return None
    except Exception:
        return None

In [0]:
# =========================================================
# 2. CLASE ORQUESTADORA (CONVERGENCIA ADAPTATIVA)
# =========================================================

class TradingOptimizer:
    """
    Clase principal que implementa Model-Based Optimization (MBO)
    con Muestreo Adaptativo y Soporte para estrategias Reversibles.
    """
    def __init__(self, data_path):
        self.data_path = data_path
        self.df_all = None
        self.config = {
            "LAMBDA": 1.3,          # Penalización por varianza
            "TRAIN_RATIO": 0.8,     # Split Temporal

            # --- PARÁMETROS ADAPTATIVOS [TFM NOTE] ---
            # Permiten dedicar más cómputo a activos difíciles y menos a los fáciles.
            "SAMPLES_INITIAL": 1000,
            "SAMPLES_STEP": 1000,
            "SAMPLES_MAX": 10000,
            "CONVERGENCE_TOL": 0.01, # Parar si mejora < 1%

            "N_POOL_PRED": 200000,
            "RANGES": {
                "X": (1, 60),
                "K": (2, 20),
                "N": (2, 20),
                "J": np.array([0.0000, 0.0002, 0.0004, 0.0006, 0.0008, 0.0010, 0.0015, 0.0020])
            },

            # ===========================
            # MEJORA AÑADIDA 1:
            # Ensemble de modelos subrogados (más estable que solo RF)
            # ===========================
            "SURROGATE_MODELS": ["RF", "GB"],   # RF + GradientBoosting
            "SURROGATE_WEIGHT_RF": 0.6,         # peso RF en el ensemble
            "SURROGATE_WEIGHT_GB": 0.4,         # peso GB en el ensemble

            # ===========================
            # MEJORA AÑADIDA 2:
            # Seguridad numérica y filtros de calidad del ajuste
            # ===========================
            "MIN_VALID_FOR_MODEL": 30,          # antes ya usabas 30, lo parametrizo
            "EPS": 1e-12,                       # evita divisiones por 0

            # ===========================
            # MEJORA AÑADIDA 3:
            # Medida extra de calidad para ranking final
            # ===========================
            "TOP10_PCT": 0.10                   # top 10%
        }
        self.rng = np.random.default_rng(42)

    def load_data(self):
        """Carga robusta de datos Parquet y limpieza."""
        print(f"📂 Cargando datos: {self.data_path}")
        try:
            df_raw = spark.read.parquet(self.data_path).toPandas()
        except Exception as e:
            raise ValueError(f"Error lectura: {e}")

        df_raw.columns = df_raw.columns.map(str).str.lower()
        col_map = {
            'timestamp': ['time', 'datetime', 'date', 'timestamp'],
            'asset_name': ['asset_name', 'symbol', 'ticker', 'asset'],
            'coste_opera_h': ['coste_opera_h', 'h', 'cost', 'coste'],
            'close': ['close', 'price', 'last', 'adj close']
        }
        found = {}
        for t, cands in col_map.items():
            m = next((c for c in df_raw.columns if c in cands), None)
            if m: found[t] = m
            elif t in ['close', 'asset_name']: raise ValueError(f"Falta columna {t}")

        self.df_all = pd.DataFrame({
            'timestamp': df_raw[found['timestamp']],
            'asset_name': df_raw[found['asset_name']],
            'close': df_raw[found['close']],
            'coste_opera_h': df_raw[found['coste_opera_h']] if 'coste_opera_h' in found else 0.0
        })

        self.df_all["timestamp"] = pd.to_datetime(self.df_all["timestamp"], utc=True).dt.tz_convert(None)
        self.df_all["close"] = pd.to_numeric(self.df_all["close"], errors="coerce")
        self.df_all["coste_opera_h"] = pd.to_numeric(self.df_all["coste_opera_h"], errors="coerce")

        self.df_all.dropna(subset=["timestamp", "close", "asset_name"], inplace=True)
        self.df_all = self.df_all[self.df_all["close"] > 1e-8]
        self.df_all = self.df_all.sort_values(["asset_name", "timestamp"]).reset_index(drop=True)
        print(f"✅ Datos listos: {len(self.df_all)} filas.")

    def _prepare_resampled_cache(self, df_asset):
        cache = {}
        x_min, x_max = self.config["RANGES"]["X"]
        df_base = df_asset.set_index("timestamp")[["close"]]
        for x in range(x_min, x_max + 1):
            df_x = df_base.resample(f"{x}min").last().dropna().reset_index()
            if len(df_x) < 50: continue
            cut = int(len(df_x) * self.config["TRAIN_RATIO"])
            cache[x] = (df_x.iloc[:cut], df_x.iloc[cut:])
        return cache

    def optimize(self):
        if self.df_all is None: self.load_data()
        assets = sorted(self.df_all["asset_name"].unique())
        results = []

        print(f"\n🚀 INICIANDO OPTIMIZACIÓN ADAPTATIVA ({len(assets)} Activos)")
        print("="*70)

        for i, asset in enumerate(assets):
            print(f"\n➡️  [{i+1}/{len(assets)}] ACTIVO: {asset}")
            t0_asset = time.time()

            mask = self.df_all["asset_name"] == asset
            df_asset = self.df_all[mask]
            h_asset = df_asset["coste_opera_h"].mean()

            print(f"   ℹ️  Filas: {len(df_asset)} | Coste (H): {h_asset:.6f}")

            # 1. Cache Resampling
            print("   ⏳ Resampling...", end=" ", flush=True)
            cache = self._prepare_resampled_cache(df_asset)
            print("Hecho.")
            if not cache: continue

            # --- BUCLE DE CONVERGENCIA [TFM NOTE] ---
            all_valid_results = []
            prev_best_score = -np.inf
            current_samples = 0

            # Piscina sintética estática para comparaciones justas
            pool_size = self.config["N_POOL_PRED"]
            pool_synthetic = np.column_stack([
                self.rng.integers(*self.config["RANGES"]["X"], pool_size),
                self.rng.integers(*self.config["RANGES"]["K"], pool_size),
                self.rng.integers(*self.config["RANGES"]["N"], pool_size),
                self.rng.choice(self.config["RANGES"]["J"], pool_size)
            ])

            best_params_global = None
            pred_score_global = -np.inf

            # ================= INICIALIZACIÓN ROBUSTA =================
            top10_real_mean = np.nan
            pearson_corr = np.nan
            spearman_corr = np.nan

            # ===========================
            # MEJORA AÑADIDA 4:
            # Guardar métricas finales del TopK para ranking
            # ===========================
            top10_real_n = 0
            top10_improvement_pct = np.nan

            while current_samples < self.config["SAMPLES_MAX"]:
                # Tamaño de lote dinámico
                if current_samples == 0:
                    n_batch_target = self.config["SAMPLES_INITIAL"]
                else:
                    n_batch_target = self.config["SAMPLES_STEP"]

                # Generamos candidatos (Sobremuestreo x4)
                n_gen = n_batch_target * 4
                params = list(zip(
                    self.rng.integers(*self.config["RANGES"]["X"], n_gen),
                    self.rng.integers(*self.config["RANGES"]["K"], n_gen),
                    self.rng.integers(*self.config["RANGES"]["N"], n_gen),
                    self.rng.choice(self.config["RANGES"]["J"], n_gen)
                ))

                # [TFM NOTE]: batch_size=256 optimiza el uso de CPU multicore
                evals = Parallel(n_jobs=-1, batch_size=256)(
                    delayed(evaluate_strategy_wrapper)(
                        p, cache, self.config["TRAIN_RATIO"],
                        1 - self.config["TRAIN_RATIO"], h_asset, self.config["LAMBDA"]
                    ) for p in params
                )

                new_valid = [r for r in evals if r is not None]
                all_valid_results.extend(new_valid)
                current_samples += n_batch_target

                n_total_valid = len(all_valid_results)

                if n_total_valid < self.config["MIN_VALID_FOR_MODEL"]:
                    print(f"   ⚠️  Pocos datos válidos ({n_total_valid})... añadiendo más.")
                    continue

                # Entrenar Modelo Subrogado
                data = np.array(all_valid_results)

                # ===========================
                # MEJORA AÑADIDA 1 (ensemble):
                # RF + GradientBoosting (sin romper tu salida)
                # ===========================
                rf = RandomForestRegressor(
                    n_estimators=200,
                    min_samples_leaf=5,
                    n_jobs=-1,
                    random_state=42
                )
                rf.fit(data[:, :4], data[:, 4])

                # GradientBoostingRegressor: solo kwargs (evita el TypeError)
                # random_state tiene que ser int, no Generator
                gb = GradientBoostingRegressor(
                    n_estimators=150,
                    learning_rate=0.05,
                    max_depth=3,
                    random_state=42
                )
                gb.fit(data[:, :4], data[:, 4])

                # ===========================
                # MEJORA AÑADIDA 5:
                # Predicción ensemble para pool_synthetic
                # ===========================
                pred_scores_rf = rf.predict(pool_synthetic)
                pred_scores_gb = gb.predict(pool_synthetic)

                w_rf = self.config["SURROGATE_WEIGHT_RF"]
                w_gb = self.config["SURROGATE_WEIGHT_GB"]

                pred_scores = (w_rf * pred_scores_rf) + (w_gb * pred_scores_gb)

                best_idx = np.argmax(pred_scores)
                current_best_score = pred_scores[best_idx]

                # Correlaciones (guardar, no imprimir)
                y_real = data[:, 4]
                # y_pred ensemble sobre train-set evaluado
                y_pred_rf = rf.predict(data[:, :4])
                y_pred_gb = gb.predict(data[:, :4])
                y_pred = (w_rf * y_pred_rf) + (w_gb * y_pred_gb)

                pearson = pearsonr(y_real, y_pred)[0]
                spearman = spearmanr(y_real, y_pred)[0]

                pearson_corr = pearson
                spearman_corr = spearman

                # =========================================================
                # MÉTRICA DE ROBUSTEZ: TOP 10% REAL
                # =========================================================

                k_top = max(1, int(self.config["TOP10_PCT"] * len(pred_scores)))
                top_idx = np.argsort(pred_scores)[-k_top:]

                # Crear lookup de scores reales
                real_scores_map = {
                    (int(x), int(k), int(n), float(j)): score
                    for x, k, n, j, score in all_valid_results
                }

                top_real_scores = []
                for p in pool_synthetic[top_idx]:
                    key = (int(p[0]), int(p[1]), int(p[2]), float(p[3]))
                    if key in real_scores_map:
                        top_real_scores.append(real_scores_map[key])

                top10_real_mean = (
                    np.mean(top_real_scores) if len(top_real_scores) > 0 else np.nan
                )
                top10_real_n = len(top_real_scores)

                # =========================================================
                # TOP-K EFFECTIVENESS (Validación del modelo subrogado)
                # =========================================================

                # Scores reales ya evaluados
                data = np.array(all_valid_results)
                scores_real = data[:, 4]   # score REAL
                params_real = data[:, :4] # (x, k, n, j)

                # 1️⃣ Selección Top 10% según score PREDICHO (pool sintético)
                k = max(1, int(0.1 * len(pred_scores)))
                top_idx = np.argsort(pred_scores)[-k:]
                top_params = pool_synthetic[top_idx]

                # 2️⃣ Lookup de scores reales
                real_score_map = {
                    (int(x), int(k), int(n), float(j)): score
                    for x, k, n, j, score in all_valid_results
                }

                scores_real_top = [
                    real_score_map[(int(p[0]), int(p[1]), int(p[2]), float(p[3]))]
                    for p in top_params
                    if (int(p[0]), int(p[1]), int(p[2]), float(p[3])) in real_score_map
                ]

                scores_real_top = np.array(scores_real_top)

                # 2️⃣ Evaluación REAL de esas estrategias (motor financiero)
                top_scores_real = []

                for params in top_params:
                    x0, k0, n0, j0 = params
                    x0 = int(x0)
                    k0 = int(k0)
                    n0 = int(n0)
                    j0 = float(j0)

                    if x0 not in cache:
                        continue

                    tr_x, _ = cache[x0]

                    pnl, ops = _calc_pnl_ops(
                        tr_x["close"].values,
                        k0, n0, j0
                    )

                    if not np.isnan(pnl) and ops > 0:
                        top_scores_real.append(pnl)

                # ===========================
                # MEJORA AÑADIDA 6:
                # Guardar mejora Top vs Global (para que el filtro y ranking tenga señal)
                # ===========================
                if len(top_scores_real) > 0:
                    mean_top = float(np.mean(top_scores_real))
                    mean_all = float(np.mean(scores_real))
                    denom = mean_all if abs(mean_all) > self.config["EPS"] else np.sign(mean_all) * self.config["EPS"]
                    top10_improvement_pct = ((mean_top / denom) - 1.0) * 100.0
                else:
                    top10_improvement_pct = np.nan

                # Calcular Mejora Relativa
                if prev_best_score <= 0:
                    pct_improve = 999.0
                else:
                    pct_improve = (current_best_score - prev_best_score) / prev_best_score

                print(f"   🔄 Iter: {current_samples} mues. | Válidas: {n_total_valid} | Score: {current_best_score:.4f} (Mejora: {pct_improve:.2%})")

                best_params_global = pool_synthetic[best_idx]
                pred_score_global = current_best_score

                # CRITERIO DE PARADA (Early Stopping)
                if pct_improve < self.config["CONVERGENCE_TOL"] and current_samples > self.config["SAMPLES_INITIAL"]:
                    print(f"   ✅ Convergencia alcanzada (< {self.config['CONVERGENCE_TOL']:.1%}). Parando.")
                    break

                prev_best_score = current_best_score

            if best_params_global is None:
                print("   ⚠️  No se encontró ninguna estrategia válida.")
                continue

            # 5. ANÁLISIS FINAL DEL GANADOR
            bx, bk, bn, bj = best_params_global
            tr_x, _ = cache[int(bx)]
            tr_ret_nominal, ops_train = _calc_pnl_ops(tr_x["close"].values, int(bk), int(bn), float(bj))

            tipo = "Inercia" if tr_ret_nominal > 0 else "Rebote"

            res = {
                "asset_name": asset,
                "best_x": int(bx),
                "best_k": int(bk),
                "best_n": int(bn),
                "best_j": bj,
                "ops_train": ops_train,
                "tipo_estrategia": tipo,
                "pred_score": pred_score_global,
                "top10_real_mean": top10_real_mean,

                # ===========================
                # MEJORA AÑADIDA 4 y 6:
                # Guardar métricas útiles para ranking final
                # ===========================
                "top10_real_n": top10_real_n,
                "top10_improvement_pct": top10_improvement_pct,

                # ya existía
                "samples_used": current_samples,

                # ===========================
                # MEJORA AÑADIDA 7:
                # Guardar correlaciones para análisis posterior (ya las calculabas)
                # ===========================
                "pearson_corr": pearson_corr,
                "spearman_corr": spearman_corr
            }
            results.append(res)

            elapsed = time.time() - t0_asset

            estado = "CANDIDATO ✅" if (res["top10_real_mean"] is not None and res["top10_real_mean"] > 0) else "DESCARTADO ❌"

            print(
                f"   🏆 FINAL [{estado}]: {tipo} | "
                f"Samples={current_samples} | "
                f"X={res['best_x']} K={res['best_k']} | "
                f"PredScore={res['pred_score']:.4f} | "
                f"Top10Real={res['top10_real_mean']:.4f}"
            )

            print(
                f"   📐 Correlación FINAL | "
                f"Pearson={pearson_corr:.3f} | "
                f"Spearman={spearman_corr:.3f}"
            )

            print(f"   ⏱️  Tiempo Activo: {elapsed:.2f}s")

        df_final = pd.DataFrame(results)

        # # 1️⃣ Umbral global de confianza del modelo
        # threshold = df_final["pred_score"].quantile(0.60)

        # # 2️⃣ Filtro duro combinado
        # df_final["pasa_filtro"] = (
        #     (df_final["top10_real_mean"] > 0) &
        #     (df_final["pred_score"] > threshold)
        # )

        # 3️⃣ Clasificación final
        def clasificar_activo(row):
          if row["pred_score"] >= 0.2:
              return "APTO FUERTE 🚀"
          elif row["pred_score"] >= 0.1:
              return "APTO DÉBIL ✅"
          else:
              return "DESCARTADO ❌"


        df_final["clasificacion"] = df_final.apply(clasificar_activo, axis=1)

        # ===========================
        # MEJORA AÑADIDA 8:
        # Orden final profesional por clasificación (orden lógico)
        # Mantengo tu sort, pero lo vuelvo determinista usando Categorical
        # ===========================
        orden_clasificacion = [
            "APTO FUERTE 🚀",
            "APTO DÉBIL ⚠️",
            "DESCARTADO ❌"
        ]
        df_final["clasificacion"] = pd.Categorical(
            df_final["clasificacion"],
            categories=orden_clasificacion,
            ordered=True
        )

        # 4️⃣ Orden final inteligente
        df_final = df_final.sort_values(
            ["clasificacion","pred_score"],
            ascending=[True, False]
        )

        return df_final


In [0]:
# =========================================================
# 3. EJECUCIÓN
# =========================================================
if __name__ == "__main__":

    print("Iniciando Sistema de Optimización (TFM)...")
    t_global = time.time()

    try:
        optimizer = TradingOptimizer(RUTA)
        df_resultados = optimizer.optimize()

        # =========================================================
        # ORDEN FINAL PROFESIONAL
        # =========================================================
        orden_clasificacion = [
            "APTO FUERTE 🚀",
            "APTO DÉBIL ⚠️",
            "DESCARTADO ❌"
        ]

        if not df_resultados.empty:

            df_resultados["clasificacion"] = pd.Categorical(
                df_resultados["clasificacion"],
                categories=orden_clasificacion,
                ordered=True
            )

            df_resultados = df_resultados.sort_values(
                by=["clasificacion", "pred_score"],
                ascending=[True, False]
            ).reset_index(drop=True)

            print("\n📊 TABLA FINAL DETALLADA:")
            print(df_resultados.to_string())

        else:
            print("⚠️ No se generaron resultados.")

    except Exception:
        print("\n❌ ERROR FATAL:")
        traceback.print_exc()

    print(
        f"\n⏱️ Tiempo Total de Ejecución: "
        f"{(time.time() - t_global)/60:.2f} minutos."
    )



Iniciando Sistema de Optimización (TFM)...
📂 Cargando datos: abfss://datos@mastertfm001sta.dfs.core.windows.net/gold/config/GOLD_Acciones_2025.parquet
✅ Datos listos: 12963643 filas.

🚀 INICIANDO OPTIMIZACIÓN ADAPTATIVA (60 Activos)

➡️  [1/60] ACTIVO: ADA-USD
   ℹ️  Filas: 525535 | Coste (H): 0.010000
   ⏳ Resampling... Hecho.
   ⚠️  Pocos datos válidos (13)... añadiendo más.


In [0]:
# =========================================================
# GUARDADO EN AZURE DATA LAKE GEN2 (Databricks)
# =========================================================

# Convertir Pandas -> Spark
spark_df = spark.createDataFrame(df_resultados)

# Guardar en ADLS (1 solo archivo CSV)
(
    spark_df
    .coalesce(1)
    .write
    .mode("overwrite")
    .option("header", True)
    .csv(estrategias_path)
)

print(f"💾 CSV guardado correctamente en ADLS: {estrategias_path}")



com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:134)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:190)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:465)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:750)
	at com.data